In [50]:
import numpy as np

In [51]:
class KalmanFilter:
    """IMPORTANT: Note that while the predict step closely matches formula (3) in the paper, the update step uses the language and concepts
    of the 'frequentist' derivation of the filter, e.g. kalman gain, innovation. Refer to Lacey's paper in Jones' article listed in the references 
    of my article. It is possible to show that the writing of the update step is equivalent to the formulas presented in the paper.
    The use of the innovation and kalman_gain allows for a leaner code (a proof of equivalence will be included asap, it can be derived by equating
    the two expressions)."""

    def __init__(self, initial_state, state_covariance, transition_matrix, observation_matrix, process_noise, measurement_noise):
        self.state = initial_state
        self.state_covariance = state_covariance
        self.transition_matrix = transition_matrix
        self.observation_matrix = observation_matrix
        self.process_noise = process_noise
        self.measurement_noise = measurement_noise

    def predict(self):
        self.state = self.transition_matrix @ self.state
        self.state_covariance = self.transition_matrix @ self.state_covariance @ self.transition_matrix.T + self.process_noise 
        
    def update(self, observation):
        predicted_observation = self.observation_matrix @ self.state
        innovation = self.observation_matrix @ self.state_covariance @ self.observation_matrix.T + self.measurement_noise
        kalman_gain = self.state_covariance @ self.observation_matrix.T @ np.linalg.inv(innovation)

        #sampling from a normal distribution
        posterior_mean = self.state + kalman_gain @ (observation - predicted_observation)
        posterior_covariance = self.state_covariance - kalman_gain @ self.observation_matrix @ self.state_covariance
        self.state = np.random.multivariate_normal(posterior_mean, posterior_covariance)
        self.state_covariance = posterior_covariance